In [ ]:
model_ocr = "./models/yolo8x_best.pt"
detector_license_plate = "./models/license_plate_detector.pt"
source = "./s3/*.jpg"


In [ ]:
from ultralytics import YOLO

model = YOLO('./models/yolo8x_best.pt')
# results = model('./s3/*.jpg')  # หรือใส่ list ของ path รูปก็ได้
results = model('1744878567335_lp0.jpg')  # หรือใส่ list ของ path รูปก็ได้

for result in results:
    result.show()  # แสดงภาพพร้อม bounding box


image 1/1 /Users/watchara/Documents/me/License-Plate-Recognition-Lab/1744878567335_lp0.jpg: 352x640 1 0, 1 2, 2 3s, 1 4, 1 A18, 1 A44, 1 BKK, 224.4ms
Speed: 0.7ms preprocess, 224.4ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)


In [5]:
label_dict = {}

# 0-9
for i in range(10):
    label_dict[str(i)] = str(i)

# A01-A44 -> ก-ฮ
for i in range(1, 45):
    key = f"A{str(i).zfill(2)}"
    value = chr(ord('ก') + i - 1)
    label_dict[key] = value

# จังหวัดย่อ
province_map = {
    "KBI": "กระบี่", "KRI": "กาญจนบุรี", "KSN": "กาฬสินธุ์", "KPT": "กำแพงเพชร", "KKN": "ขอนแก่น",
    "CTI": "จันทบุรี", "CCO": "ฉะเชิงเทรา", "CBI": "ชลบุรี", "CNT": "ชัยนาท", "CPM": "ชัยภูมิ",
    "CPN": "ชุมพร", "CRI": "เชียงราย", "CMI": "เชียงใหม่", "TRG": "ตรัง", "TRT": "ตราด",
    "TAK": "ตาก", "NYK": "นครนายก", "NPT": "นครปฐม", "NPM": "นครพนม", "NMA": "นครราชสีมา",
    "NRT": "นครศรีธรรมราช", "NSN": "นครสวรรค์", "NBI": "นนทบุรี", "NWT": "นราธิวาส", "NAN": "น่าน",
    "BKN": "บึงกาฬ", "BRM": "บุรีรัมย์", "PTE": "ปทุมธานี", "PKN": "ประจวบคีรีขันธ์", "PRI": "ปราจีนบุรี",
    "PTN": "ปัตตานี", "PYO": "พะเยา", "AYA": "พระนครศรีอยุธยา", "PNA": "พังงา", "PLG": "พัทลุง",
    "PCT": "พิจิตร", "PLK": "พิษณุโลก", "PBI": "เพชรบุรี", "PNB": "เพชรบูรณ์", "PRE": "แพร่",
    "PKT": "ภูเก็ต", "MKM": "มหาสารคาม", "MDH": "มุกดาหาร", "MSN": "แม่ฮ่องสอน", "YST": "ยโสธร",
    "YLA": "ยะลา", "RET": "ร้อยเอ็ด", "RNG": "ระนอง", "RYG": "ระยอง", "RBR": "ราชบุรี",
    "LRI": "ลพบุรี", "LPG": "ลำปาง", "LPN": "ลำพูน", "LEI": "เลย", "SSK": "ศรีสะเกษ",
    "SNK": "สกลนคร", "SKA": "สงขลา", "STN": "สตูล", "SPK": "สมุทรปราการ", "SKM": "สมุทรสงคราม",
    "SKN": "สมุทรสาคร", "SKW": "สระแก้ว", "SRI": "สระบุรี", "SBR": "สิงห์บุรี", "STI": "สุโขทัย",
    "SPB": "สุพรรณบุรี", "SNI": "สุราษฎร์ธานี", "SRN": "สุรินทร์", "NKI": "หนองคาย", "NBP": "หนองบัวลำภู",
    "ATG": "อ่างทอง", "ACR": "อำนาจเจริญ", "UDN": "อุดรธานี", "UTT": "อุตรดิตถ์", "UTI": "อุทัยธานี",
    "UBN": "อุบลราชธานี", "BKK": "กรุงเทพมหานคร"
}

label_dict.update(province_map)

In [ ]:
def decode_license_plate(result, class_names, label_dict, province_map):
    from operator import itemgetter

    boxes = result.boxes
    cls_list = boxes.cls.cpu().numpy()
    x_centers = boxes.xywh[:, 0].cpu().numpy()

    detections = [(class_names[int(cls)], float(x)) for cls, x in zip(cls_list, x_centers)]
    sorted_detections = sorted(detections, key=itemgetter(1))

    decoded = ""
    province = ""

    for cls_name, _ in sorted_detections:
        if cls_name in province_map:
            province = label_dict.get(cls_name, f"[{cls_name}]")
        else:
            decoded += label_dict.get(cls_name, f"[{cls_name}]")

    return decoded + province

In [6]:
from operator import itemgetter

# โหลดโมเดล และรันภาพ
model = YOLO('./models/yolo8x_best.pt')
results = model('1744878567335_lp0.jpg')

# คลาสทั้งหมดตามที่เทรนไว้
class_names = model.names  # เช่น {0: '0', 1: '1', ..., 55: 'BKK'}

for result in results:
    boxes = result.boxes
    cls_list = boxes.cls.cpu().numpy()       # class id เช่น [0, 2, 35, ...]
    x_centers = boxes.xywh[:, 0].cpu().numpy()  # ค่าศูนย์กลางแนวนอนของกล่อง

    # รวมเป็น tuple (class_name, x_center)
    detections = [(class_names[int(cls)], float(x)) for cls, x in zip(cls_list, x_centers)]

    # เรียงจากซ้ายไปขวา
    sorted_detections = sorted(detections, key=itemgetter(1))

    # แปลงคลาสเป็นข้อความ readable ด้วย label_dict
    decoded = ""
    for cls_name, _ in sorted_detections:
        decoded += label_dict.get(cls_name, f"[{cls_name}]")  # fallback เผื่อ class ไม่อยู่ใน dict

    print("ทะเบียนรถที่ถอดรหัสได้:", decoded)



image 1/1 /Users/watchara/Documents/me/License-Plate-Recognition-Lab/1744878567335_lp0.jpg: 352x640 1 0, 1 2, 2 3s, 1 4, 1 A18, 1 A44, 1 BKK, 217.1ms
Speed: 0.9ms preprocess, 217.1ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)
ทะเบียนรถที่ถอดรหัสได้: 3ฒฬกรุงเทพมหานคร2430


In [ ]:
from ultralytics import YOLO
import cv2
import glob
import os

# โหลดโมเดล
detector_lp = YOLO('./models/license_plate_detector.pt')
ocr_model = YOLO('./models/yolo8x_best.pt')

# โหลดภาพทั้งหมดจาก ./s3/*.jpg
image_paths = glob.glob('./s3/*.jpg')

for img_path in image_paths:
    img = cv2.imread(img_path)

    # Step 1: ตรวจจับป้ายทะเบียน
    results = detector_lp(img)
    for i, box in enumerate(results[0].boxes.xyxy):
        x1, y1, x2, y2 = map(int, box[:4])
        lp_crop = img[y1:y2, x1:x2]

        # Optional: save cropped license plate
        crop_filename = f"./crops/{os.path.basename(img_path).split('.')[0]}_lp{i}.jpg"
        os.makedirs("./crops", exist_ok=True)
        cv2.imwrite(crop_filename, lp_crop)

        # Step 2: ใช้ OCR กับภาพที่ตัดมา
        ocr_results = ocr_model(lp_crop)

        # แสดงผลลัพธ์
        print(f"[{img_path}] → Plate {i+1}")
        # ocr_results[0].show()


0: 640x384 (no detections), 38.7ms
Speed: 1.7ms preprocess, 38.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 license_plate, 46.6ms
Speed: 1.7ms preprocess, 46.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 320x640 1 0, 1 2, 1 3, 1 6, 1 9, 1 A18, 1 A25, 1 BKK, 208.9ms
Speed: 0.7ms preprocess, 208.9ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)
[./s3/1741078372064.jpg] → Plate 1

0: 640x384 1 license_plate, 29.0ms
Speed: 2.4ms preprocess, 29.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 256x640 (no detections), 279.4ms
Speed: 0.6ms preprocess, 279.4ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 640)
[./s3/1743477981295.jpg] → Plate 1

0: 640x384 1 license_plate, 33.4ms
Speed: 1.6ms preprocess, 33.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 320x640 1 0, 1 2, 1 3, 1 6, 1 A07, 1 A22, 1 BKK, 471.3ms
Speed: 0.7ms preprocess, 471.3ms infer

In [3]:
from ultralytics import YOLO
import cv2
import numpy as np
import glob
import os

def imflatfield(I, sigma):
    A = I.astype(np.float32) / 255
    Ihsv = cv2.cvtColor(A, cv2.COLOR_BGR2HSV)
    A = Ihsv[:, :, 2]

    filterSize = int(2 * np.ceil(2 * sigma) + 1)
    shading = cv2.GaussianBlur(A, (filterSize, filterSize), sigma, borderType=cv2.BORDER_REFLECT)
    meanVal = np.mean(A)
    shading = np.maximum(shading, 1e-6)
    B = A * meanVal / shading
    Ihsv[:, :, 2] = B
    B = cv2.cvtColor(Ihsv, cv2.COLOR_HSV2BGR)
    B = np.round(np.clip(B * 255, 0, 255)).astype(np.uint8)
    return B

# โหลดโมเดล
detector_lp = YOLO('./models/license_plate_detector.pt')
ocr_model = YOLO('./models/yolo8x_best.pt')

# โหลดภาพทั้งหมดจาก ./s3/*.jpg
image_paths = glob.glob('./s3/*.jpg')
os.makedirs("./crops", exist_ok=True)

for img_path in image_paths:
    img = cv2.imread(img_path)

    # Step 1: ตรวจจับป้ายทะเบียน
    results = detector_lp(img)
    for i, box in enumerate(results[0].boxes.xyxy):
        x1, y1, x2, y2 = map(int, box[:4])
        lp_crop = img[y1:y2, x1:x2]

        # Step 2: Preprocess ด้วย imflatfield และ denoise
        flat_img = imflatfield(lp_crop, sigma=15)
        denoised = cv2.fastNlMeansDenoisingColored(flat_img, None, 3, 3, 7)

        # บันทึก cropped + cleaned image (optional)
        base = os.path.basename(img_path).split('.')[0]
        crop_path = f"./crops2/{base}_lp{i}.jpg"
        cv2.imwrite(crop_path, denoised)

        # Step 3: ใช้ OCR
        ocr_results = ocr_model(denoised)

        # Step 4: แสดงผล (เฉพาะตำแหน่ง และ confidence)
        # Step 4: แสดงผล (ตำแหน่ง + confidence)
        print(f"[{img_path}] → Plate {i+1}")
        for box in ocr_results[0].boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            print(f" - Char class index: {cls}, Confidence: {conf:.2f}")


        # Optional: แสดงภาพผลลัพธ์
        # ocr_results[0].show()



0: 640x384 (no detections), 31.6ms
Speed: 2.2ms preprocess, 31.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 license_plate, 20.0ms
Speed: 0.9ms preprocess, 20.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)

0: 320x640 1 0, 1 2, 1 3, 1 6, 1 9, 1 A18, 1 A25, 1 BKK, 190.0ms
Speed: 0.8ms preprocess, 190.0ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)
[./s3/1741078372064.jpg] → Plate 1
 - Char class index: 23, Confidence: 0.96
 - Char class index: 0, Confidence: 0.95
 - Char class index: 2, Confidence: 0.94
 - Char class index: 30, Confidence: 0.94
 - Char class index: 6, Confidence: 0.92
 - Char class index: 9, Confidence: 0.92
 - Char class index: 3, Confidence: 0.90
 - Char class index: 51, Confidence: 0.80

0: 640x384 1 license_plate, 18.0ms
Speed: 0.9ms preprocess, 18.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 256x640 1 0, 2 1s, 1 A18, 1 A22, 190.5ms
Speed: 0.7ms preprocess, 1

KeyboardInterrupt: 